# Naive Classifier for Fake news recognition

## Packages

In [ ]:
library(tidyverse)
library(ggplot2)
library(stopwords)
library(tidytext)
library(stringr)
library(purrr)
library(magrittr)
library(parallel)

In [ ]:
numCores <- detectCores()

# Reading the data

### Kaggle dataset

In [ ]:
train.filename <- 'dataset/archive/train.csv'
test.filename <- 'dataset/archive/test.csv'

In [ ]:
train.df <- read.delim(train.filename, sep = ',')
test.df <- read.delim(test.filename, sep = ',')

### Second dataset

# Display the train and test datasets

In [ ]:
head(train.df)

In [ ]:
head(test.df)

# Splitting the datasets

In [ ]:
# shuffle
set.seed(29062023)
train.df=train.df[sample(1:nrow(train.df), replace = F),] # shuffle the dataset

In [ ]:
# set splitting percentages
perc_split <- 0.8  # set splitting percentages
n_rows <- nrow(train.df)
n_rows_train <- floor(n_rows * perc_split)

In [ ]:
# splitting
train.df.train <- train.df[1:n_rows_train, ]
train.df.validation <- train.df[-c(1:n_rows_train), ]

In [ ]:
cat('Total rows:',nrow(train.df),
    '\nTrain rows:',nrow(train.df.train),
    '\nValidation rows',nrow(train.df.validation))

## Histogram of labels

In [ ]:
breaks <- seq(from=-0.5, to=5.5, by=1)
labels <- c('Barely True', 'False', 'Half-True', 'Mostly-True', 'Not known', 'True')
colors <- c('#FF6666','#FF9966', '#FFCC66', '#99FF99' , '#CCCCCC','#00FF99')

classes<-hist(x = train.df.train$Labels, 
     breaks = breaks, 
     labels = labels,
     main = 'Histogram of classes',
     xlab = 'Classes',
     ylab = 'Count in training set',
     col = colors
    )

In [ ]:
n_classes <- length(labels)

## Defining the prior of each classes

In [ ]:
prior.classes<-classes$density

## Stop words

In [ ]:
as.list(get_stopwords())$word

# Cleaning function

In [ ]:
cleaning_tokens <- function(token.df){
    word <- ifelse(endsWith(token.df[,1], "s"), substr(token.df[,1], 1, nchar(token.df[,1]) - 1), token.df[,1])
    word <- ifelse(endsWith(word, "'"), substr(word, 1, nchar(word) - 1), word)
    word <- ifelse(str_detect(word, "[0-9]+"), '--number--', word)
    return(as.data.frame(word))
}

# Vocabulary

In [ ]:
tags_numbers <- list(tags = c('Barely True', 'False', 'Half-True', 'Mostly-True', 'Not known', 'True'),
                     numbers = c(0:5))

### Vocabulary before feature selection

In [ ]:
vocabulary <- function(train.df.train, tags_numbers){
    #take in input a df with 'Text' column containing the messages and 'tag_numbers' the names of the labels
    
    myframes <- list()

    for (i in 1:length(tags_numbers$tags)) {
        tag <- tags_numbers$tags[i]

        train.df.train %>%
        filter(Labels == tags_numbers$numbers[i]) %>%
        select(Text) %>%
        unnest_tokens(word, Text) %>%
        anti_join(get_stopwords(), by = join_by(word)) %>%
        cleaning_tokens %>%
        arrange(word) %>%
        group_by(word) %>%
        reframe(counts = n()) %>%
        arrange(desc(counts)) %>%
        setNames(c("word", tag)) -> myframes[[i]]
    } 
    counts<-purrr::reduce(myframes, dplyr::full_join, by = 'word')
    counts %<>% replace(is.na(.),0)
    return(counts)
}

# Feature Selection 

## Rankings

### Ranking per class
Ranks by a certain metric (df_rank) per class

In [ ]:
ranking_per_class <- function(df_rank, counts, n_classes, frac = 0.5) {
    mywords <- list()
    for (i in 2:(n_classes + 1)) {
        df_rank[, c(1, i)] %>%
            arrange(desc(.[[2]])) %>%
            top_frac(frac, .[[2]]) %>%
            select(word) -> mywords[[i]]
    }
    words <- purrr::reduce(mywords, rbind)
    words %<>%
        unique

    counts %>%
        right_join(words["word"], by = join_by(word)) -> naive.bayes.vocabulary

    return(naive.bayes.vocabulary)
}

### Ranking by mean of classes
Ranks by a certain metric (df_rank) by mean

In [ ]:
ranking_per_mean <- function(df_rank, counts, n_classes, frac = 0.5) {
    df_rank["mean"] <- rowMeans(df_rank[2:(n_classes + 1)])

    df_rank %>%
        arrange(desc(mean)) %>%
        top_frac(frac, mean) %>%
        select(word) -> vocabulary.train.features

    counts %>%
        right_join(vocabulary.train.features["word"], by = join_by(word)) -> naive.bayes.vocabulary

    return(naive.bayes.vocabulary)
}

# Frequency

## Frequency by mean 

In [ ]:
feature_selection.frequency_mean <- function(counts, n_classes, frac = 0.5) {
    counts %>%
        select(-word) %>%
        colSums() -> tot_counts_per_class

    counts_prob <- cbind(counts["word"], counts[2:(n_classes + 1)]/tot_counts_per_class)
    
    return(ranking_per_mean(counts_prob, counts, n_classes, frac))
}

## Frequency per class

In [ ]:
feature_selection.frequency_per_class <- function(counts, n_classes, frac = 0.5) {
    return(ranking_per_class(counts, counts, n_classes, frac))
}

# Chi squared

In [ ]:
word_chi_squared <- function(word, train.df.train, n_classes) {
    if (word != "--number--") {
        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(grepl(word, Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> yw_c

        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(!grepl(word, Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> nw_c
    } else {
        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(grepl("[0-9]+", Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> yw_c

        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(!grepl("[0-9]+", Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> nw_c
    }

    chi <- vector(length = n_classes)
    for (i in 1:n_classes) {
        n11 <- yw_c[yw_c$Labels == (i - 1), ]$c
        n10 <- sum(yw_c[yw_c$Labels != (i - 1), ]$c)
        n01 <- nw_c[yw_c$Labels == (i - 1), ]$c
        n00 <- sum(nw_c[nw_c$Labels != (i - 1), ]$c)

        chi[i] <- ((n11 + n10 + n01 + n00) * (n11 * n00 - n10 * n01)^2)/((n11 + n01) *
            (n11 + n10) * (n10 + n00) * (n01 + n00))
    }
    chi[is.na(chi)] = 0

    return(chi)

}

In [ ]:
vocabulary_chi_squared <- function(counts,  train.df.train, n_classes, numCores=numCores) {
    
    df <- mclapply(X = counts$word, FUN = word_chi_squared, train.df.train = train.df.train,
        n_classes = n_classes, mc.cores = numCores)

    chi <- cbind(counts[, 1], as.data.frame(do.call(rbind, df)))
    
    return(chi)
}

## Chi by mean

In [ ]:
feature_selection.chi_squared_per_mean <- function(counts, train.df.train, n_classes,
    frac = 0.5) {

    chi<- vocabulary_chi_squared(counts, train.df.train, n_classes, numCores)

    return(ranking_per_mean(chi, counts, n_classes, frac))
}

## Chi by class

In [ ]:
feature_selection.chi_squared_per_class <- function(counts, train.df.train, n_classes,
    frac = 0.5) {

    chi <- vocabulary_chi_squared(counts, train.df.train, n_classes, numCores)

    return(ranking_per_class(chi, counts, n_classes, frac))
}

# Mutual information

## Mutual information per class

In [ ]:
word_mutual_info <- function(word, train.df.train, n_classes) {
    if (word != "--number--") {
        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(grepl(word, Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> yw_c

        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(!grepl(word, Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> nw_c
    } else {
        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(grepl("[0-9]+", Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> yw_c

        train.df.train %>%
            group_by(Labels, .drop = FALSE) %>%
            filter(!grepl("[0-9]+", Text, ignore.case = TRUE)) %>%
            summarise(c = n()) -> nw_c
    }

    MI <- vector(length = n_classes)
    for (i in 1:n_classes) {
        n11 <- yw_c[yw_c$Labels == (i - 1), ]$c
        n10 <- sum(yw_c[yw_c$Labels != (i - 1), ]$c)
        n01 <- nw_c[yw_c$Labels == (i - 1), ]$c
        n00 <- sum(nw_c[nw_c$Labels != (i - 1), ]$c)
        n1_ <- n11 + n10
        n0_ <- n01 + n00
        n_1 <- n01 + n11
        n_0 <- n10 + n00
        n <- n11 + n01 + n10 + n00
        MI[i] <- n11/n * log2(n * n11/(n1_ * n_1)) + n01/n * log2(n * n01/(n0_ *
            n_1)) + n10/n * log2(n * n10/(n1_ * n_0)) + n00/n * log2(n * n00/(n0_ *
            n_0))
    }
    MI[is.na(MI)] = 0

    return(MI)

}

In [ ]:
vocabulary_mutual_info <- function(counts,  train.df.train, n_classes, numCores=numCores) {
    
    df <- mclapply(X = counts$word, FUN = word_mutual_info, train.df.train = train.df.train,
        n_classes = n_classes, mc.cores = numCores)

    mutual_info <- cbind(counts[, 1], as.data.frame(do.call(rbind, df)))
    
    return(mutual_info)
}

## Mutual by class

In [ ]:
feature_selection.mutual_info_per_class <- function(counts, train.df.train, n_classes,
    frac = 0.5) {

    mutual_info <- vocabulary_mutual_info(counts, train.df.train, n_classes, numCores)

    return(ranking_per_class(mutual_info, counts, n_classes, frac))
}

## Mutual by mean

In [ ]:
feature_selection.mutual_info_per_mean <- function(counts, train.df.train, n_classes,
    frac = 0.5) {

    mutual_info <- vocabulary_mutual_info(counts, train.df.train, n_classes, numCores)

    return(ranking_per_mean(mutual_info, counts, n_classes, frac))
}

# Likelihood computation

In [ ]:
likelihood.denominator <- function(naive.bayes.vocabulary) {
    naive.bayes.vocabulary %>%
        select(-word) %>%
        colSums() -> tot_counts_per_class

    den <- tot_counts_per_class + length(naive.bayes.vocabulary[[1]])
    return(den)
}

In [ ]:
likelihood <- function(naive.bayes.vocabulary, den) {
    likelihood.token <- cbind(naive.bayes.vocabulary["word"], (naive.bayes.vocabulary[2:(n_classes +
        1)] + 1)/den)
    return(likelihood.token)
}

## Nayve Bayes

In [ ]:
tweet_tokenize <- function(msg) {
    tib <- data.frame(msg)
    colnames(tib) <- "tweet"
    tib %>%
        unnest_tokens(word, tweet) %>%
        anti_join(get_stopwords(), by = join_by(word)) %>%
        cleaning_tokens -> token.list
    return(token.list)
}

In [ ]:
bayes.probability <- function(token.list, likelihood.token, prior.classes, den, n_classes) {

    query_out <- log(as.numeric(prior.classes))
    for (i in 1:length(token.list)) {
        query <- log(as.numeric(likelihood.token[likelihood.token["word"] == token.list$word[i]][2:(n_classes +
            1)]))
        if (is.na(query[1])) {
            # because if the word is not found, all NA are returned. if the
            # word is in the vocabulary the proper likelihood is returned
            query <- as.numeric(log(1/den))
        }
        query_out <- query_out + query
    }
    query_out <- exp(query_out)
    query_out <- query_out/sum(query_out)
    return(query_out)
}

In [ ]:
naive.bayes <- function(msg, likelihood.token, prior.classes, den, n_classes) {
    token.list <- as.vector(tweet_tokenize(msg))
    probability <- bayes.probability(token.list, likelihood.token, prior.classes,
        den, n_classes)
    predicted_class <- which.max(probability) - 1
    return(predicted_class)
}

# Training: creating and saving the dfs with words and countings according to feature selection method

In [ ]:
fraction<-seq(from =0.1, to =1, by=0.2)

In [ ]:
# Return data.frame with only counted and tokenized words
counts <- vocabulary(train.df.train, tags_numbers)

In [ ]:
for (i in seq_along(fraction)) {
    write.csv(feature_selection.frequency_per_class(counts, n_classes, frac = fraction[i]),
        file = paste("models/frequency_per_class", fraction[i], ".csv", sep = ""))
    write.csv(feature_selection.frequency_mean(counts, n_classes, frac = fraction[i]),
        file = paste("models/frequency_mean", fraction[i], ".csv", sep = ""))
    write.csv(feature_selection.chi_squared_per_class(counts, n_classes, frac = fraction[i]),
        file = paste("models/chi_per_class", fraction[i], ".csv", sep = ""))
    write.csv(feature_selection.chi_squared_per_mean(counts, n_classes, frac = fraction[i]),
        file = paste("models/chi_per_mean", fraction[i], ".csv", sep = ""))
    write.csv(feature_selection.mutual_info_per_class(counts, n_classes, frac = fraction[i]),
        file = paste("models/mutual_per_class", fraction[i], ".csv", sep = ""))
    write.csv(feature_selection.mutual_info_per_mean(counts, n_classes, frac = fraction[i]),
        file = paste("models/mutual_per_mean", fraction[i], ".csv", sep = ""))
}

# Run all the functions

In [ ]:
# acc_per_class <- vector()
# for (i in seq_along(fraction)) {
#     # Vocabulary optimized with feature selection
#     naive.bayes.vocabulary <- feature_selection.ranking_per_class(counts, n_classes,
#         frac = fraction[i])

#     # Compute denominator of likelihood
#     denominator <- likelihood.denominator(naive.bayes.vocabulary)

#     # Compute the likelihood for each tokinez and unique word
#     likelihood.token <- likelihood(naive.bayes.vocabulary, denominator)

#     # Run the prediction
#     predicted_classes <- mclapply(X = train.df.validation$Text, FUN = naive.bayes,
#         likelihood.token = likelihood.token, prior.classes = prior.classes, den = denominator,
#         n_classes = n_classes, mc.cores=numCores)

#     # Check % of right predictions
#     acc_per_class[i] <- sum(predicted_classes == train.df.validation$Labels)/length(train.df.validation$Labels)
# }

In [ ]:
# acc_per_class <- vector()
# for (i in seq_along(fraction)) {
#     # Vocabulary optimized with feature selection
#     naive.bayes.vocabulary <- feature_selection.ranking_per_class(counts, n_classes,
#         frac = fraction[i])

#     # Compute denominator of likelihood
#     denominator <- likelihood.denominator(naive.bayes.vocabulary)

#     # Compute the likelihood for each tokinez and unique word
#     likelihood.token <- likelihood(naive.bayes.vocabulary, denominator)

#     # Run the prediction
#     predicted_classes <- mclapply(X = train.df.validation$Text, FUN = naive.bayes,
#         likelihood.token = likelihood.token, prior.classes = prior.classes, den = denominator,
#         n_classes = n_classes, mc.cores=numCores)

#     # Check % of right predictions
#     acc_per_class[i] <- sum(predicted_classes == train.df.validation$Labels)/length(train.df.validation$Labels)
# }

In [ ]:
# acc_freq_mean <- vector()
# for (i in seq_along(fraction)) {
#     # Vocabulary optimized with feature selection
#     naive.bayes.vocabulary <- feature_selection.frequency_mean(counts, n_classes,
#         frac = fraction[i])

#     # Compute denominator of likelihood
#     denominator <- likelihood.denominator(naive.bayes.vocabulary)

#     # Compute the likelihood for each tokinez and unique word
#     likelihood.token <- likelihood(naive.bayes.vocabulary, denominator)

#     # Run the prediction
#     predicted_classes <- mclapply(X = train.df.validation$Text, FUN = naive.bayes,
#         likelihood.token = likelihood.token, prior.classes = prior.classes, den = denominator,
#         n_classes = n_classes, mc.cores=numCores)

#     # Check % of right predictions
#     acc_freq_mean[i] <- sum(predicted_classes == train.df.validation$Labels)/length(train.df.validation$Labels)
# }

In [ ]:
# plot(fraction, acc_per_class, type='l', ylim = c(0,1))
# lines(fraction, acc_freq_mean, col='red')
# legend(0, 1, legend = c("freq mean", "top per class"), col = c("red", "black"), lty = 1)